# Redes Neuronais + Backpropagation

In [1]:
from pathlib import Path
from itertools import product
from collections import Counter

import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

### Read Data

In [2]:
data = pd.read_csv('./data_tp1', header=None)

In [3]:
features = data.loc[:, 1:]
target = data.loc[:, 0]

In [4]:
features.shape, target.shape

((5000, 784), (5000,))

In [5]:
Counter(target)

Counter({1: 571,
         2: 530,
         9: 520,
         7: 512,
         4: 500,
         3: 500,
         8: 489,
         6: 462,
         0: 460,
         5: 456})

### Model Hiperparameters Options [[docs](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)]

In [6]:
activation = 'tanh' # Sigmoid

neuron_input = 784
neuron_output = 10

neurons_hidden = [25, 50, 100]
solver = 'sgd'
batch_sizes = [
    10, 50,         # Mini Batch
    1,              # Stochastic Gradient Descent
    target.shape[0] # Gradient Descent
]

learning_rate_inits = [0.5, 1, 10]

In [7]:
params = list()
for batch_size, learning_rate_init, neuron_hidden in product(batch_sizes, learning_rate_inits, neurons_hidden):
    params.append({
        'activation': activation,
        'hidden_layer_sizes': neuron_hidden,
        'solver': solver,
        'batch_size': batch_size,
        'learning_rate': 'constant',
        'learning_rate_init': learning_rate_init,
        'early_stopping': False,
        'nesterovs_momentum': True,
        'momentum': .9,
        'verbose': False,
        'alpha': .0001,
        'max_iter': 256,  # number of epochs
        'shuffle': True,
        'random_state': None,
        'n_iter_no_change': 10,
        'tol': 1e-4,
    })

In [8]:
relevant_params = ['batch_size', 'learning_rate_init', 'hidden_layer_sizes']

In [9]:
len(params)

36

### Model Instance

In [ ]:
reports = list()
for curr_params in tqdm(params):
    report = dict()
    report['params'] = curr_params

    model = MLPClassifier(**curr_params)
    model.fit(features, target)

    metrics = classification_report(
        y_pred=model.predict(features),
        y_true=target,
        output_dict=True,
        zero_division=0,
    )

    report['metrics'] = metrics
    report['curves'] = {
        'loss_curve': model.loss_curve_,
        'best_loss': model.best_loss_,
        'n_iter': model.n_iter_
    }

    reports.append(report)

### Evaluate Results

In [10]:
fpath = 'results.csv'
if Path(fpath).exists():
    results = pd.read_csv(fpath)
else:
    results = pd.json_normalize(reports)
    results.to_csv(fpath, index=False)

In [11]:
params_cols = ['metrics.accuracy'] + [f'params.{param}' for param in relevant_params]
results[params_cols].sort_values('metrics.accuracy', ascending=False).head(10)

,metrics.accuracy,params.batch_size,params.learning_rate_init,params.hidden_layer_sizes
27,0.6810,5000,0.5,25
28,0.6384,5000,0.5,50
30,0.5650,5000,1.0,25
29,0.2846,5000,0.5,100
11,0.2118,50,0.5,100
35,0.2062,5000,10.0,100
31,0.1886,5000,1.0,50
15,0.1710,50,10.0,25
32,0.1430,5000,1.0,100
10,0.1350,50,0.5,50


In [12]:
(
    results
    [results['curves.best_loss'] < 10]
    .select_dtypes('number')
    .mean()
)

params.hidden_layer_sizes           58.333333
params.batch_size                 5000.000000
params.learning_rate_init            3.833333
params.momentum                      0.900000
params.alpha                         0.000100
params.max_iter                    256.000000
params.random_state                       NaN
params.n_iter_no_change             10.000000
params.tol                           0.000100
metrics.0.precision                  0.389719
metrics.0.recall                     0.380193
metrics.0.f1-score                   0.296949
metrics.0.support                  460.000000
metrics.1.precision                  0.497592
metrics.1.recall                     0.615100
metrics.1.f1-score                   0.536630
metrics.1.support                  571.000000
metrics.2.precision                  0.224847
metrics.2.recall                     0.477358
metrics.2.f1-score                   0.264343
metrics.2.support                  530.000000
metrics.3.precision               

---